In [1]:
import pandas as pd

uid_check = ['아파트1-104-1006']
tss_transition = pd.DataFrame(columns=['K','tss','wss', 'ecv', 'cdpv', 'mdis' ,'mcdpv', 'og_length', 'new_length', 'sequence'])

In [3]:
import pandas as pd
from modules.UCKMeans import UCKMeans as UC
from modules.UCOSKMeans import UCOSKMeans as UCOS
from modules.SortMeans import SortMeans as SM

# Mongo 연결
from pymongo import MongoClient as mc
import random
import pprint as pp

#  print(uid_check)
# custom module
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

mongo_uri = "mongodb://localhost:27017"
client = mc(mongo_uri)

keti_pr_db = client.keti_pattern_recognition
household_col = keti_pr_db.household_info

# while True:
#     total = household_col.find({"uid": { "$nin" : uid_check }}).count()
#     db_datas = list(household_col.\
#                     find({
#                         "uid": { "$nin" : uid_check }
#                         }).skip(random.randrange(0,total)).limit(1))
#     if len(db_datas) == 0 :
#         continue
#     else:
#         db_datas = db_datas[0]
#         break;
        
db_datas = household_col.\
                find_one({
                    "uid": "아파트1-103-106"
                })

# require module
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime as dt

sns.set(rc={'figure.figsize': (15.7, 13.27)})
plt.rcParams['figure.figsize'] = 15.7,13.27
plt.rcParams['font.family'] = 'AppleGothic'

# custom module
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import custom.supports as sp
import custom.mapgenerator as mg
import custom.dateutils as du

SEASONS = {
    "봄" : [3, 4, 5],
    "여름" : [6, 7, 8],
    "가을" : [9, 10, 11],
    "겨울" : [12, 1, 2]
}

SEASONSARR = ['봄','여름','가을','겨울']

'''
 0 : 봄
 1 : 여름
 2 : 가을
 3 : 겨울
'''

DAYARR = ["월", "화", "수", "목", "금", "토", "일"]

uid, timeslot = db_datas['uid'], db_datas['timeslot']
uid_check.append(uid)

print("{} pattern recognition".format(uid))


datelist = [
    dt.strptime(ts['time'], "%Y-%m-%d T%H:%M %z").date()
    for ts in timeslot
]
datelist = list(set(datelist))
datelist.sort()

ts_datas = {}
start_idx = 0
end_idx = 96
enl = 1

for date in datelist:
    ts_datas[date] = [ ts['power'] * enl for ts in timeslot[start_idx:end_idx]]
    start_idx = end_idx
    end_idx = end_idx + 96

ts_datas = pd.DataFrame(ts_datas).T
datas = ts_datas.reset_index().copy()

datas.rename(columns={"index": "date"}, inplace=True)
datas['date'] = pd.to_datetime(datas['date'])
datas['month'] = [dt.month for dt in datas['date']]
datas = [
    datas[(datas['month'].isin(SEASONS[season]))].copy()
    for season in SEASONSARR
]

bak_datas = datas.copy()

season_datas = {
    "봄": datas[0].copy(),
    "여름": datas[1].copy(),
    "가을": datas[2].copy(),
    "겨울": datas[3].copy()
}

# merge timeslot
season = "가을"

timeslot = 1
merge_size = timeslot * 4
merge_datas = season_datas[season].copy()
merge_datas = merge_datas[
    merge_datas.columns.difference(['month', 'date'])
]
merge_datas.index = season_datas[season]['date']

tmp = pd.DataFrame()
start_idx = 0
end_idx = 96
for date in merge_datas.index:
    in_arr = []
    og_arr = merge_datas.loc[date].values
    for merging in range(0, int(end_idx / merge_size)):
        in_arr.append(og_arr[
            (merging) * merge_size
            :
            (merging + 1) * merge_size
        ].sum())
#     logging
#     print("date end : {}, size: {}".format(date, len(in_arr)))

    tmp[date] = in_arr
merge_datas = tmp.copy()

merge_datas

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
아파트1-103-106 pattern recognition


,2018-09-01,2018-09-02,2018-09-03,2018-09-04,2018-09-05,2018-09-06,2018-09-07,2018-09-08,2018-09-09,2018-09-10,...,2018-11-21,2018-11-22,2018-11-23,2018-11-24,2018-11-25,2018-11-26,2018-11-27,2018-11-28,2018-11-29,2018-11-30
0,0.187,0.231,0.244,0.174,0.250,0.288,0.222,0.284,0.201,0.168,...,0.217,0.213,0.228,0.242,0.250,0.258,0.213,0.217,0.224,0.204
1,0.289,0.225,0.229,0.230,0.294,0.183,0.174,0.200,0.175,0.201,...,0.236,0.224,0.249,0.241,0.229,0.329,0.208,0.182,0.224,0.215
2,0.270,0.240,0.273,0.292,0.228,0.219,0.225,0.189,0.183,0.194,...,0.215,0.221,0.213,0.206,0.243,0.252,0.198,0.211,0.207,0.224
3,0.220,0.377,0.243,0.275,0.190,0.198,0.170,0.214,0.213,0.170,...,0.218,0.210,0.233,0.228,0.257,0.223,0.220,0.223,0.214,0.220
4,0.160,0.266,0.214,0.209,0.265,0.235,0.206,0.185,0.290,0.180,...,0.245,0.221,0.246,0.238,0.301,0.242,0.198,0.316,0.221,0.220
5,0.220,0.218,0.259,0.194,0.178,0.187,0.190,0.191,0.174,0.199,...,0.222,0.311,0.207,0.222,0.296,0.247,0.195,0.207,0.217,0.235
6,0.214,0.306,0.333,0.227,0.220,0.205,0.181,0.206,0.206,0.212,...,0.200,0.260,0.241,0.217,0.236,0.228,0.216,0.221,0.215,0.301
7,0.209,0.205,0.713,0.742,1.011,0.384,1.076,0.183,0.193,0.744,...,0.839,0.772,0.881,0.269,0.251,0.712,1.003,0.779,0.706,0.811
8,0.171,0.251,1.297,1.168,1.098,0.982,0.786,0.185,0.165,0.834,...,1.052,1.013,1.005,0.336,0.258,1.144,0.900,0.904,1.018,0.905
9,0.206,1.041,1.224,0.913,0.828,1.027,0.770,0.329,0.187,0.760,...,0.890,0.989,1.072,0.240,0.222,0.906,0.882,0.808,0.847,0.969


In [27]:
from modules.THMeans_2 import THMeans as tm
tm(merge_datas).run()

---Season Means---
---init TSS Check---
- calc TSS(Total Sum Of Squares) success!
---Dimension Reduction---
[]
---Remove Outliers---
- dis_check: 1.0904563424409432, sim_check: -0.053543301729650716
- remove outlier success: 91 => 91
- calc TSS(Total Sum Of Squares) success!
---K Setting 7 ---
---7:Clustering Start---
---Now 1---
---First Cluster Group Init---
---Init Cluster---
---First K Is Mean Pattern---
---Rest K Select---
Test Start!
arr length: 91
Cluster Worst Pattern In
1: calc... [90, 0]
1: [90, 0, 45]
2: calc... [90, 45, 0]
2: calc... [90, 45, 0, 68]
2: [90, 45, 0, 68, 23]
4: calc... [90, 68, 45, 23, 0]
4: calc... [90, 68, 45, 23, 0, 79]
4: calc... [90, 68, 45, 23, 0, 79, 57]
4: calc... [90, 68, 45, 23, 0, 79, 57, 34]
4: [90, 68, 45, 23, 0, 79, 57, 34, 12]
init_cluster [90, 79, 68, 57, 45, 23]
-1: K Setting Okay
-2: K Setting Okay
-3: K Setting Okay
-4: K Setting Okay
-5: K Setting Okay
-6: K Setting Okay
{0: array([0.2269011 , 0.22484615, 0.22805495, 0.23152747, 0.22857143,